In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline 
from sklearn import linear_model 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn import feature_selection
from sklearn import metrics
import plotly
import kaggle
import scipy
import statsmodels.api as sm
import statsmodels.formula.api as smf

le = LabelEncoder()

In [9]:


### Let's import our data
train_data = pd.read_csv('data/train.csv',index_col='PassengerId')
test_data = pd.read_csv('data/test.csv',index_col='PassengerId')
test_data_survived = pd.read_csv('data/gender_submission.csv',index_col='PassengerId')
### and test if everything OK


In [9]:
test_data.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [10]:
test_data = test_data.join(test_data_survived)

In [11]:
test_data.describe()

,Pclass,Age,SibSp,Parch,Fare,Survived
count,418.000000,332.000000,418.000000,418.000000,417.000000,418.000000
mean,2.265550,30.272590,0.447368,0.392344,35.627188,0.363636
std,0.841838,14.181209,0.896760,0.981429,55.907576,0.481622
min,1.000000,0.170000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,21.000000,0.000000,0.000000,7.895800,0.000000
50%,3.000000,27.000000,0.000000,0.000000,14.454200,0.000000
75%,3.000000,39.000000,1.000000,0.000000,31.500000,1.000000
max,3.000000,76.000000,8.000000,9.000000,512.329200,1.000000


In [12]:
train_data.describe()


,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [12]:
train_data["Cabin_cc"] = le.fit_transform(train_data["Cabin"])
train_data["Sex_cc"] = le.fit_transform(train_data["Sex"])
train_data["Embarked_cc"] = le.fit_transform(train_data["Embarked"])
train_data = train_data.dropna()

X_train = train_data.drop(["Survived", "Name", "Cabin","Sex", "Embarked","Ticket" ],axis=1)
Y_train = train_data.Survived 


In [13]:
test_data["Cabin_cc"] = le.fit_transform(test_data["Cabin"])
test_data["Sex_cc"] = le.fit_transform(test_data["Sex"])
test_data["Embarked_cc"] = le.fit_transform(test_data["Embarked"])
test_data = test_data.dropna()

X_test = test_data.drop(["Survived", "Name", "Cabin","Sex", "Embarked","Ticket" ],axis=1)
Y_test = test_data["Survived"]

AttributeError: 'Series' object has no attribute 'Survived'

In [14]:
results = sm.OLS(Y_train.values.tolist(), X_train.values.tolist()).fit()
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.693
Model:                            OLS   Adj. R-squared (uncentered):              0.679
Method:                 Least Squares   F-statistic:                              49.30
Date:                Mon, 13 Dec 2021   Prob (F-statistic):                    5.90e-41
Time:                        21:31:35   Log-Likelihood:                         -115.36
No. Observations:                 183   AIC:                                      246.7
Df Residuals:                     175   BIC:                                      272.4
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [15]:
X_train.head()

,Pclass,Age,SibSp,Parch,Fare,Cabin_cc,Sex_cc,Embarked_cc
PassengerId,,,,,,,,
2,1,38.0,1,0,71.2833,81,0,0
4,1,35.0,1,0,53.1000,55,0,2
7,1,54.0,0,0,51.8625,129,1,2
11,3,4.0,1,1,16.7000,145,0,2
12,1,58.0,0,0,26.5500,49,0,2


In [16]:
X_train_limited = X_train.drop(["SibSp","Parch","Cabin_cc","Embarked_cc"], axis= 1)
X_test_limited = X_test.drop(["SibSp","Parch","Cabin_cc","Embarked_cc"], axis= 1)

In [17]:
X_train_limited.head()

,Pclass,Age,Fare,Sex_cc
PassengerId,,,,
2,1,38.0,71.2833,0
4,1,35.0,53.1000,0
7,1,54.0,51.8625,1
11,3,4.0,16.7000,0
12,1,58.0,26.5500,0


In [20]:
results = sm.OLS(Y_train.values.tolist(), X_train_limited.values.tolist()).fit()
print(results.summary())

AttributeError: 'DataFrame' object has no attribute 'params'

In [19]:
X_train_limited.head()

,Pclass,Age,Fare,Sex_cc
PassengerId,,,,
2,1,38.0,71.2833,0
4,1,35.0,53.1000,0
7,1,54.0,51.8625,1
11,3,4.0,16.7000,0
12,1,58.0,26.5500,0


In [ ]:
Need to change python version to 3.7

In [213]:
#results = sm.OLS(Y_train.values.tolist(), X_train_limited.values.tolist()).fit_regularized(method='sqrt_lasso',L1_wt=1)
#print(results.summary())

In [53]:
lasso_params = {
    'selector__k':[1,2,3,4,5,6,7],
    'fit__alpha':[1.0,0.02, 0.024, 0.025, 0.026, 0.03],
    'fit__fit_intercept':[True, False],
    'fit__max_iter':[100,1000,10000]
    }
ridge_params = {
    'selector__k':[1,2,3,4,5,6,7],
    'fit__alpha':[200, 230, 250,265, 270, 275, 290, 300, 500],
    'fit__solver':['svd', 'cholesky', 'lsqr', 'sag'],
    'fit__fit_intercept':[True, False],
    'fit__normalize':[True, False]
    }

linear_params = {
    'selector__k':[1,2,3,4,5,6,7],
    'fit__fit_intercept': [True, False]
}

In [54]:
pipeline_baseline = Pipeline([
    ('fit', linear_model.LinearRegression())
])

pipeline_linear = Pipeline([
    ('selector', feature_selection.SelectKBest(feature_selection.f_regression)),
    ('fit', linear_model.LinearRegression())
])

pipeline_lasso = Pipeline([
    ('selector', feature_selection.SelectKBest(feature_selection.f_regression)),
    ("fit", linear_model.Lasso())
])

pipeline_ridge = Pipeline([
    ('selector', feature_selection.SelectKBest(feature_selection.f_regression)),
    ('fit',linear_model.Ridge())
])


In [37]:
print("OLS")
ols = pipeline_baseline.fit(X_train, Y_train)
ols.score(X_test, Y_test)

OLS


0.5373501766558703

In [38]:
print("ols _2 ")
grid_ols = GridSearchCV( estimator=pipeline_linear, param_grid=linear_params, scoring='r2')
gsols = grid_ols.fit(X_train_limited, Y_train)
gsols.best_estimator_.named_steps['fit'].coef_


ols _2 


/home/tristan/anaconda3/envs/dataAnaBase/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
30 fits failed out of a total of 70.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/home/tristan/anaconda3/envs/dataAnaBase/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/tristan/anaconda3/envs/dataAnaBase/lib/python3.9/site-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/tristan/anaconda3/envs/dataAnaBase/lib/python3.9/site-packages/sklearn/pipeline.py"

array([-0.004848  , -0.47223889])

In [39]:
print(gsols.best_params_)
print(gsols.best_estimator_.score(X_test_limited, Y_test))
print(gsols.best_estimator_.__dict__)

{'fit__fit_intercept': True, 'selector__k': 2}
0.5929297437242145
{'steps': [('selector', SelectKBest(k=2, score_func=<function f_regression at 0x7f9058814940>)), ('fit', LinearRegression())], 'memory': None, 'verbose': False}


In [55]:
grid_lasso = GridSearchCV( estimator=pipeline_lasso, param_grid=lasso_params, scoring='r2')
gsLaso = grid_lasso.fit(X_train_limited, Y_train)


/home/tristan/anaconda3/envs/dataAnaBase/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
540 fits failed out of a total of 1260.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
180 fits failed with the following error:
Traceback (most recent call last):
  File "/home/tristan/anaconda3/envs/dataAnaBase/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/tristan/anaconda3/envs/dataAnaBase/lib/python3.9/site-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/tristan/anaconda3/envs/dataAnaBase/lib/python3.9/site-packages/sklearn/pipeline

In [56]:
print(gsLaso.best_params_)
print(gsLaso.best_estimator_.score(X_test_limited, Y_test))
print(gsLaso.best_estimator_.named_steps['selector'].pvalues_)
print(gsLaso.best_estimator_.named_steps['fit'].coef_)


{'fit__alpha': 0.02, 'fit__fit_intercept': True, 'fit__max_iter': 100, 'selector__k': 2}
0.4988826716215681
[6.42493848e-01 5.18950331e-04 7.00255771e-02 8.69386832e-15]
[-0.00525441 -0.38977434]


In [33]:
grid_ridge = GridSearchCV(estimator = pipeline_ridge, param_grid=ridge_params,scoring='r2')
gsRidge = grid_ridge.fit(X_train, Y_train)

/home/tristan/anaconda3/envs/dataAnaBase/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(
/home/tristan/anaconda3/envs/dataAnaBase/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing st

In [57]:
print(gsRidge.best_params_)
print(gsRidge.best_estimator_.score(X_test, Y_test))

{'fit__alpha': 200, 'fit__fit_intercept': True, 'fit__normalize': False, 'fit__solver': 'svd', 'selector__k': 5}
0.0234079778220887
